In [2]:
! pip install sister==0.1.10

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 48.9 MB/s eta 0:00:00
  Using cached pybind11-2.12.0-py3-none-any.whl (234 kB)
  Created wheel for sister: filename=sister-0.1.10-py3-none-any.whl size=5380 sha256=71fe798ba55ab50b5d356db88f75cb24a7573c1f057a1fb16f4c886ee4a1840b
  Stored in directory: /root/.cache/pip/wheels/89/d2/02/d2ba3090f304c3c18e7287b46a1786e03316fb8741e2098eb2
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4227142 sha256=bcb0406034df2bdd9d5b0df85e6e878d8d34fa7e49a2313f0d8c53942e66b99c
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built sister fasttext


In [3]:
import pandas as pd
import ast
from google.colab import drive
import sister
from scipy.spatial import distance

drive.mount('/content/drive')

Mounted at /content/drive


### Read data

In [5]:
data = pd.read_csv('test_with_description.csv')

In [6]:
pred_gpt = pd.read_json(path_or_buf='/content/drive/MyDrive/Text2Graph/text2graph_test_with_answers_gpt4_answer_desc_all_posteval.jsonl', lines=True)

### Extract answer from GPT CoT reasoning by spliting marker "So the answer is:"

In [7]:
for j in range(len(data)):
  cur_ques = data.loc[j, 'question_id']
  for k in range(len(pred_gpt)):
    if k == cur_ques:
       data.loc[j, 'pred_gpt'] = pred_gpt.loc[k, 'predText']

In [8]:
for i in range(len(data)):
  data.loc[i, 'gpt_answer'] = data.loc[i, 'pred_gpt'].split('the answer is: ')[-1]

In [9]:
for i in range(len(data)):
  if '**' in data.loc[i, 'gpt_answer']:
    data.loc[i, 'gpt_answer'] = data.loc[i, 'gpt_answer'].replace('**', '')

In [10]:
ques_ids = set()
for i in range(len(data)):
  if data.loc[i, 'gpt_answer'] == data.loc[i, 'answerEntity']:
    data.loc[i, 'prediction'] = 1
    ques_ids.add(data.loc[i, 'question_id'])

### Replace '.' in the end of gpt answer for the exact match

In [11]:
for i in range(len(data)):
  cur_answer = data.loc[i, 'gpt_answer']
  cur_length = len(data.loc[i, 'gpt_answer'])
  if cur_answer[-1] == ".":
    cur_answer = cur_answer[:cur_length-1]
  data.loc[i, 'gpt_answer'] = cur_answer

In [12]:
for i in range(len(data)):
  if data.loc[i, 'gpt_answer'] == data.loc[i, 'answerEntity']:
    data.loc[i, 'prediction'] = 1
    ques_ids.add(data.loc[i, 'question_id'])

In [13]:
for i in range(len(data)):
  if data.loc[i, 'question_id'] in ques_ids:
    if pd.isna(data.loc[i, 'prediction']):
      data.loc[i, 'prediction'] = 0

In [14]:
nonmatched_answers = []

count = 0
for k in range(1000):
  if data.query('question_id == @k').prediction.sum() == 0:
    nonmatched_answers.append(k)

### Iterate to match multiple possible answers

In [15]:
to_remove = []
for i in range(len(data)):
  if data.loc[i, 'question_id'] in nonmatched_answers:
    gpt_answers = data.loc[i, 'gpt_answer'].split(', ')
    for k in range(len(gpt_answers)):
      if gpt_answers[k] == data.loc[i, 'answerEntity']:
        data.loc[i, 'prediction'] = 1
        to_remove.append(data.loc[i, 'question_id'])

In [16]:
to_remove = list(set(to_remove))

In [17]:
for i in range(len(data)):
  if data.loc[i, 'question_id'] in to_remove:
    if pd.isna(data.loc[i, 'prediction']):
      data.loc[i, 'prediction'] = 0

In [18]:
nonmatched_answers = []

count = 0
for k in range(1000):
  if data.query('question_id == @k').prediction.sum() == 0:
    nonmatched_answers.append(k)

### Check non-exact match

In [19]:
for i in range(len(data)):
  if data.loc[i, 'question_id'] in nonmatched_answers:
    if data.loc[i, 'answerEntity'] in data.loc[i, 'gpt_answer']:
      data.loc[i, 'prediction'] = 1
    else:
      data.loc[i, 'prediction'] = 0

In [20]:
nonmatched_answers = []

count = 0
for k in range(1000):
  if data.query('question_id == @k').prediction.sum() == 0:
    nonmatched_answers.append(k)

In [21]:
len(nonmatched_answers)

25

### Fix numerous identical answers

In [22]:
nonmatched_answers = []

count = 0
for k in range(1000):
  if data.query('question_id == @k').prediction.sum() > 1:
    nonmatched_answers.append(k)

In [23]:
len(nonmatched_answers)

163

### Estimate length of subgraph

In [24]:
for i in range(len(data)):
  data.loc[i, 'n_links']= len(ast.literal_eval(data.loc[i, 'graph'])['links'])

### Answers that should not be fixed (if there are multiple answers to the question and they are not identical)

In [25]:
to_remove = []
for i in range(1000):
  test_subset = data.query('question_id == @i and prediction == 1')
  if len(test_subset) == len(test_subset.answerEntity.unique()) and i in nonmatched_answers:
    to_remove.append(i)

In [26]:
to_remove

[9, 68, 267, 290, 522, 533, 547, 556, 619, 631, 648, 658, 669, 685, 926, 981]

In [27]:
nonmatched_answers = [elem for elem in nonmatched_answers if elem not in to_remove]

In [28]:
len(nonmatched_answers)

147

### Select among the multiple identical answers the ones with the minimal subgraph[Текст ссылки](https://)

In [29]:
to_remove = []
for i in range(len(nonmatched_answers)):
  cur_ques = nonmatched_answers[i]
  min_links = data.query('question_id == @cur_ques and prediction == 1').n_links.min()
  test_subset = data.query('question_id == @cur_ques and prediction == 1 and n_links != @min_links')
  test_inds = test_subset.index
  if len(test_subset.answerEntity.unique()) == 1:
    for k in range(len(test_inds)):
      data.loc[test_inds[k], 'prediction'] = 0
      to_remove.append(cur_ques)

In [30]:
nonmatched_answers = [elem for elem in nonmatched_answers if elem not in to_remove]

In [31]:
len(nonmatched_answers)

37

### Select among the multiple identical answers the ones where cosine similarity of the mean fastText embeddings between question and answer is maximum

In [32]:
sentence_embedding = sister.MeanEmbedding(lang="en")

Loading model...


In [33]:
to_remove = []
for i in range(len(nonmatched_answers)):
  cur_ques = nonmatched_answers[i]
  test_subset = data.query('question_id == @cur_ques and prediction == 1').reset_index(drop=True)
  sent_emb = sentence_embedding(test_subset.loc[0, 'question'])
  best_cos_sim = 0
  best_ind = 0
  for k in range(len(test_subset)):
    if pd.notna(test_subset.loc[k, 'answer_desc']):
      cur_desc = sentence_embedding(test_subset.loc[k, 'answer_desc'])
      if 1-distance.cosine(sent_emb, cur_desc) > best_cos_sim:
        best_cos_sim = 1-distance.cosine(sent_emb, cur_desc)
        best_ind = k
  test_inds = list(data.query('question_id == @cur_ques and prediction == 1').index)
  for q in range(len(test_inds)):
    data.loc[test_inds[q], 'prediction'] = 0
  data.loc[test_inds[best_ind], 'prediction'] = 1

### Look at questions with multiple answers

In [34]:
for i in range(1000):
  cur_ques = i
  if data.query('question_id == @cur_ques').prediction.sum() > 1:
    print(i)

9
68
72
267
290
522
533
547
556
619
631
648
658
669
685
834
926
981


In [41]:
data.query('question_id == 547 and prediction == 1')

,sample_id,question,questionEntity,answerEntity,questionEntityId,answerEntityId,graph,question_id,answer_desc,pred_gpt,gpt_answer,prediction,n_links
5994,5994,Which country joined European Union on the 1st...,European Union,Cyprus,Q458,Q2967757,"{'nodes': [{'type': 'INTERNAL', 'name_': 'Q617...",547.0,Roman provincia,To find out which country joined the European ...,"Cyprus, Hungary, Poland, Slovakia, Slovenia",1.0,3.0
5997,5997,Which country joined European Union on the 1st...,European Union,Hungary,Q458,Q28,"{'nodes': [{'type': 'QUESTIONS_ENTITY', 'name_...",547.0,country in Central Europe,To find out which country joined the European ...,"Cyprus, Hungary, Poland, Slovakia, Slovenia",1.0,3.0
5999,5999,Which country joined European Union on the 1st...,European Union,Poland,Q458,Q36,"{'nodes': [{'type': 'QUESTIONS_ENTITY', 'name_...",547.0,country in Central Europe,To find out which country joined the European ...,"Cyprus, Hungary, Poland, Slovakia, Slovenia",1.0,3.0
6000,6000,Which country joined European Union on the 1st...,European Union,Slovakia,Q458,Q214,"{'nodes': [{'type': 'QUESTIONS_ENTITY', 'name_...",547.0,country in Central Europe,To find out which country joined the European ...,"Cyprus, Hungary, Poland, Slovakia, Slovenia",1.0,3.0
6002,6002,Which country joined European Union on the 1st...,European Union,Slovenia,Q458,Q215,"{'nodes': [{'type': 'QUESTIONS_ENTITY', 'name_...",547.0,country in Central Europe,To find out which country joined the European ...,"Cyprus, Hungary, Poland, Slovakia, Slovenia",1.0,3.0


### Check amount of ones and zeros in the final submit

In [42]:
data.prediction.value_counts()

prediction
0.0    9949
1.0    1012
Name: count, dtype: int64

In [43]:
9949+1012 == len(data)

True

### Create final submission file

In [44]:
data[['sample_id', 'prediction']].to_csv("post_eval_answer_desc.tsv", sep="\t", index=False)